In [ ]:
import os
import pandas as pd
import re
import copy

def search_files(directory):
    """Ищет все файлы внутри директории
    input: строка (начальная директория)
    output: список (все файлы, содержащиеся в данной директории)
    """

    return [os.path.join(dp, f) for dp,dn,filenames in os.walk(directory) for f in filenames]

def files_filter(files, extension):
    """Фильтрует файлы согласно их разрешения
    input: список (файлы для фильтрации), строка - расширение файла
    output: список (отфильтрованные согласно требуемого расширения файлы)
    """

    return [file_ for file_ in files if file_.lower().endswith(extension)]

def clean_df(string):
    return re.sub(r'\n',' ',string)

def excel_to_csv(excel_files_list):
    """Преобразовывает excel-файлы в csv-файлы (с ними удобней в дальнейшем вести работу)
    input: список (excel-файлы, которые необходимо преобразовать)
    """

    for excel_file in excel_files_list:
        try:
            xlsx = pd.ExcelFile(excel_file)
            res_sheets = filter(lambda x: x!='Worksheet',xlsx.sheet_names)
            for sheet in res_sheets:
                try:
                    df = pd.read_excel(xlsx, sheet)
                    columns_names = df.columns.tolist()
                    for column in columns_names:
                        df[column] = df[column].astype(str)
                        df[column] = df[column].apply(clean_df)
                    res_csv_file = '.'.join(excel_file.split('.')[:-1])+str(sheet)+'.csv'#
                    df.to_csv(res_csv_file, sep=';', encoding = 'utf-8')
                    print("ready"+str(res_csv_file))
                except Exception as error:
                    print("Error2:{0}\nFile:{1}".format(str(error),str(excel_file)))
                    continue
        except Exception as error:
            print("Error1:{0}\nFile:{1}".format(str(error),str(excel_file)))
            continue          

def prepare_csv(csv_files):
    """Чистит csv-файлы до строк заголовков
    input: список (csv-файлы, для которых необходимо произвести очистку)
    """
    csv_vocab = {}
    for csv_file in csv_files:
        try:
            text = ""
            with open(csv_file, 'r', encoding='utf-8') as file_1:
                text = file_1.read()
                lines = text.split('\n')
                for i,line in enumerate(lines):
                    if 'стоимост' in line.lower() or 'итог' in line.lower() or 'сумм' in line.lower():
                        text = '\n'.join(lines[i:])
                        break
            with open(csv_file, 'w', encoding='utf-8') as file_2:
                file_2.write(text)
            csv_vocab[csv_file] = text    
        except:
            continue
    return csv_vocab

def search_ids(df, column_number):
    """Формирование датафрейма исходя из НЖЯ
    input: датафрейм, номер колонки с НЖЯ
    output: отфильтрованный датафрейм
    """

    col_names = df.columns.tolist()
    res_df = df[df[col_names[column_number-1]].notnull()]
    return res_df

Следующая ячейка запускается только один раз (первый запуск) и преобразовывает excel-файлы к формату csv

In [ ]:
all_files = search_files(directory) # нашли все файлы внутри папки
xlsx_files = files_filter(all_files,'xlsx') # нашли файлы с расширением xlsx
xls_files = files_filter(all_files,'xls') # нашли файлы с расширением xls
all_excel_files = xlsx_files+xls_files # объединили списки в один
excel_to_csv(all_excel_files) # преобразовали все excel-файлы в csv

In [ ]:
nzyas_frame = pd.read_excel('NZYA_QUICK.xlsx') #загрузка файла с НЖЯ

In [ ]:
# выбор директории с файлами для обработки
directory = "csv"

In [ ]:
csv_files =  files_filter(search_files(directory), 'csv') #выбрать только csv-файлы

In [ ]:
vocab = prepare_csv(csv_files) #формируем словарь текстовых значений для каждого файла

In [ ]:
"""
Поиск sd или id с помощью регулярного выражения (чтобы исключить совпадения подстроки)
"""

def search_id_qw(sd, line): 
    status  =False
    if sd=="nan":
        return status
    if len(re.findall(r';\s*(sd|)\s*'+str(sd)+'\s*;',line))>0:
        status =True
    return status

In [ ]:
nzya_frame = search_ids(nzyas_frame, 3).iloc[:,:] #выбираем интервал из нашего файла для проверки

In [ ]:
nzya_frame.head() # просмотр выгруженного файла с НЖЯ

In [ ]:
"""
Очищает строки от буквенных значений (в дальнейшем ускоряет процесс поиска с помощью регулярного выражения)
input: строка для преобразования
output: очищенная от буквенных значений строка
"""

def clean_text(string):
    clean_string = re.sub(r'[a-zа-я]','',string)
    return clean_string

In [ ]:
vocab_keys = vocab.keys()
new_vocab = {key: vocab[key].lower().split('\n') for key in vocab_keys}
clean_vocab = {key: clean_text(vocab[key].lower()).split('\n') for key in vocab_keys}
print("Ready")
first_sd_file_name = 'nzya_quick2.csv' # название первого файла, куда будут писаться строки заголовка и подтянутые sd id
files_paths_file_name = 'nzya_quick2_files.csv' # название второго файла, куда будет писаться информация по найденным sd (расположение файла,sd,id)
clean_vocab_copy = copy.copy(clean_vocab)
with open (first_sd_file_name,'w', encoding='utf-8') as file1, open(files_paths_file_name,'w',encoding='utf-8') as file2:
    for index,row in nzya_frame.iterrows():
        col_1 = str(row[0])#[2:] # срез строки
        col_2 = str(row[1])
        for key in vocab_keys:
            status = 0
            for i, line in enumerate(clean_vocab[key]):
                if search_id_qw(col_1,line) or search_id_qw(col_2,line):
                    fl = new_vocab[key][0]
                    file1.write('{0}\n{1}\n|\n'.format(fl,line))
                    print('{0}:{1}\n{2}\n{3}\n\n'.format(col_1,col_2,fl,str(line)))
                    file2.write('{0};{1};{2}\n'.format(key,col_1,col_2))
                    clean_vocab_copy[key].remove(line)
                    status = 1
                    break
            clean_vocab = copy.copy(clean_vocab_copy)
            if status==1:
                break